<a href="https://colab.research.google.com/github/IAT-ComputationalCreativity-Spring2025/Week11-Genetic-Algorithms/blob/main/DEAP_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to DEAP

Documentation: [https://deap.readthedocs.io/en/master/]

## Imports

In [ ]:
# Install required libraries
! pip install deap numpy matplotlib

# Import necessary libraries
import random
import numpy as np
import matplotlib.pyplot as plt

from deap import base, creator, tools, algorithms

## DEAP Components

DEAP has several key components:
1. creator - for creating new classes with specific attributes
2. base.Toolbox - for registering operations
3. tools - contains built-in operators for genetic algorithms
4. algorithms - contains ready-to-use evolutionary algorithms

Let's create the following:

- FitnessMax: A fitness class that maximizes a single objective (weight=1.0)
- Individual: A class that inherits from list and has FitnessMax as its fitness attribute

In [ ]:
# First, let's define a fitness class that maximizes a single objective
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# Then, we define an individual type that inherits from list and has our fitness attribute
creator.create("Individual", list, fitness=creator.FitnessMax)

# Initialize the toolbox
toolbox = base.Toolbox()

## Register Operations

In [ ]:
# Register a function to create binary genes (0 or 1)
toolbox.register("attr_bool", random.randint, 0, 1)

# Register a function to create individuals (lists of genes)
# tools.initRepeat repeatedly calls attr_bool to create a list
# The third parameter defines the length of the individual (10 genes in this case)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, 10)

# Register a function to create a population (list of individuals)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Create and display a sample individual and population
sample_individual = toolbox.individual()
sample_population = toolbox.population(n=3)

print("Sample individual:", sample_individual)
print("Sample population:", sample_population)

## Evaluation function

In [ ]:
# Define a simple evaluation function that counts the number of 1s in the individual
def evalOneMax(individual):
    """Count the number of 1s in the individual (maximize this)"""
    return sum(individual),  # Note the comma - this creates a tuple

# Register the evaluation function in the toolbox
toolbox.register("evaluate", evalOneMax)

# Test the evaluation function
test_individual = creator.Individual([1, 0, 1, 1, 0, 0, 1, 1, 1, 0])
fitness = evalOneMax(test_individual)
print(f"Individual: {test_individual}")
print(f"Fitness: {fitness}")

## Register Genetic Operators

In [ ]:
# Register genetic operators in the toolbox

# Crossover operator - two-point crossover
toolbox.register("mate", tools.cxTwoPoint)

# Mutation operator - flip bits with a 10% probability for each bit
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)

# Selection operator - tournament selection with tournament size 3
toolbox.register("select", tools.selTournament, tournsize=3)

# Let's test these operators

# Create two test individuals
ind1 = creator.Individual([1, 1, 0, 0, 1, 1, 0, 0, 1, 1])
ind2 = creator.Individual([0, 0, 1, 1, 0, 0, 1, 1, 0, 0])

print("Before crossover:")
print(f"Individual 1: {ind1}")
print(f"Individual 2: {ind2}")

# Apply crossover
child1, child2 = toolbox.mate(ind1, ind2)
print("\nAfter crossover:")
print(f"Child 1: {child1}")
print(f"Child 2: {child2}")

# Apply mutation to child1
toolbox.mutate(child1)
print("\nAfter mutation:")
print(f"Mutated Child 1: {child1}")

## Create Initial Population

In [ ]:
# Create initial population
population = toolbox.population(n=50)

# Evaluate the entire population
fitnesses = list(map(toolbox.evaluate, population))
for ind, fit in zip(population, fitnesses):
    ind.fitness.values = fit

# Get some statistics about the initial population
fits = [ind.fitness.values[0] for ind in population]

print(f"Initial population size: {len(population)}")
print(f"Min fitness: {min(fits)}")
print(f"Max fitness: {max(fits)}")
print(f"Average fitness: {sum(fits) / len(fits)}")

## Run the Algorithm with Statistics

In [ ]:
# Set up statistics to track
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", min)
stats.register("max", max)
stats.register("std", np.std)

# Parameters
N_GENS = 40  # Number of generations
P_CROSS = 0.5  # Crossover probability
P_MUTATE = 0.2  # Mutation probability

# Lists to store statistics
gen_numbers = []
max_fitness = []
avg_fitness = []

# Run the algorithm manually so we can track progress
for gen in range(N_GENS):
    # Select the next generation individuals
    offspring = toolbox.select(population, len(population))
    
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))
    
    # Apply crossover and mutation
    for i in range(1, len(offspring), 2):
        if random.random() < P_CROSS:
            offspring[i-1], offspring[i] = toolbox.mate(offspring[i-1], offspring[i])
            del offspring[i-1].fitness.values
            del offspring[i].fitness.values
    
    for i in range(len(offspring)):
        if random.random() < P_MUTATE:
            offspring[i], = toolbox.mutate(offspring[i])
            del offspring[i].fitness.values
    
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    
    # Replace the old population by the offspring
    population[:] = offspring
    
    # Gather statistics
    fits = [ind.fitness.values[0] for ind in population]
    gen_numbers.append(gen)
    max_fitness.append(max(fits))
    avg_fitness.append(sum(fits) / len(fits))
    
    # Print statistics
    if gen % 5 == 0:
        print(f"Generation {gen}: Max = {max(fits)}, Avg = {sum(fits)/len(fits)}")

# Print final best solution
best_ind = tools.selBest(population, 1)[0]
print("\nBest individual:")
print(f"Genes: {best_ind}")
print(f"Fitness: {best_ind.fitness.values[0]}")

## Visualize the Evolution Progress

In [ ]:
# Plot the evolution progress
plt.figure(figsize=(10, 6))
plt.plot(gen_numbers, max_fitness, 'r-', label='Max Fitness')
plt.plot(gen_numbers, avg_fitness, 'b-', label='Average Fitness')
plt.xlabel('Generation')
plt.ylabel('Fitness')
plt.title('Evolution Progress')
plt.legend()
plt.grid(True)
plt.show()

## Using Built-in Algorithms

In [ ]:
# Instead of manually implementing the evolutionary loop,
# we can use DEAP's built-in algorithms

# Reset the population
population = toolbox.population(n=50)

# Use eaSimple algorithm with the same parameters
result, logbook = algorithms.eaSimple(population, toolbox, 
                                     cxpb=P_CROSS, mutpb=P_MUTATE, 
                                     ngen=N_GENS, stats=stats, 
                                     verbose=True)

# Plot statistics from the logbook
gen = logbook.select("gen")
fit_max = logbook.select("max")
fit_avg = logbook.select("avg")

plt.figure(figsize=(10, 6))
plt.plot(gen, fit_max, 'r-', label='Max Fitness')
plt.plot(gen, fit_avg, 'b-', label='Average Fitness')
plt.xlabel('Generation')
plt.ylabel('Fitness')
plt.title('Evolution Progress (using eaSimple)')
plt.legend()
plt.grid(True)
plt.show()

# Print final best solution
best_ind = tools.selBest(result, 1)[0]
print("\nBest individual from eaSimple:")
print(f"Genes: {best_ind}")
print(f"Fitness: {best_ind.fitness.values[0]}")